In [33]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import fetch_california_housing#　加利福尼亚房屋价值数据
from sklearn.metrics import mean_squared_error # mse均方误差
from sklearn.metrics import mean_absolute_error #mae绝对均值误差
from sklearn.metrics import r2_score

In [9]:
house = fetch_california_housing()
X = pd.DataFrame(house.data, columns=house.feature_names)
Y = house.target

In [11]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


特征解释      
MedInc：该街区住户的收入中位数        
HouseAge：该街区房屋使用年代的中位数       
AveRooms：该街区平均的房间数目    
AveBedrms：该街区平均的卧室数目       
Population：街区人口      
AveOccup：平均入住率   
Latitude：街区的纬度     
Longitude：街区的经度

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_predict_train = lr.predict(X_train)
y_predict_test = lr.predict(X_test)

In [22]:
print(lr.score(X_train, y_train))
print(lr.score(X_test, y_test))

0.6112941337977223
0.5926087785518779


## 回归类模型评估标准

### MSE均方误差
sklearn中使用RSS的变体，均方误差MSE（mean squared error）来衡量我们的预测值和真实值的差异：

$$
MSE = \frac{1}{m}\sum_{i=1}^{m}({y_i} - {\hat{y}_i})^2
$$

在sklearn当中，我们有两种方式调用这个评估指标，一种是使用sklearn专用的模型评估模块metrics里的类mean_squared_error，另一种是调用交叉验证的类cross_val_score并使用里面的scoring参数来设置使用均方误差。

In [24]:
mean_squared_error(y_predict_test, y_test)

0.5431489670037236

In [25]:
# 交叉验证
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [27]:
cross_val_score(lr, X, Y, cv=5,scoring="neg_mean_squared_error").mean()

-0.5582901717686546

我们看到,均方误差都是负的值

虽然均方误差,从公式角度来看应该永远为正，

但是sklearn中的参数scoring下，均方误差作为评判标准时，却是计算”负均方误差“（neg_mean_squared_error）。

这是因为sklearn在计算模型评估指标的时候，会考虑指标本身的性质，均方误差本身是一种误差，所以被sklearn划分为模型的一种损失(loss)。

在sklearn当中，所有的损失都使用负数表示，因此均方误差也被显示为负数了。真正的均方误差MSE的数值，其实就是neg_mean_squared_error去掉负号的数字。 

### MAE绝对均值误差
除了MSE，我们还有与MSE类似的MAE（Mean absolute error，绝对均值误差）：

$$
MAE = \frac{1}{m}\sum_{i=0}^{m-1}|{y_i} - {\hat{y}_i}|
$$

其表达的概念与均方误差完全一致，不过在真实标签和预测值之间的差异外我们使用的是L1范式（绝对值）。

使用中，MSE和MAE选一个来使用就好了。

在sklearn当中，我们使用命令
from sklearn.metrics import mean_absolute_error来调用MAE，同时，我们也可以使用交叉验证中的
scoring = "neg_mean_absolute_error"，以此在交叉验证时调用MAE。

In [30]:
mean_absolute_error(y_predict_test, y_test)

0.5361818140641827

In [31]:
cross_val_score(lr, X, Y, cv=5, scoring="neg_mean_absolute_error").mean()

-0.5474961907866818

## 是否拟合了足够的信息

对于回归类算法而言，只探索数据预测是否准确是不足够的。除了数据本身的数值大小之外，我们还希望我们的模型能够捕捉到数据的”规律“，比如数据的分布规律，单调性等等，而是否捕获了这些信息并无法使用MSE来衡量。

![](https://pictes.oss-cn-beijing.aliyuncs.com/%E5%BE%AE%E8%AF%BE%20-%20sklearn/week%209%20Linear%20Model/MSE.PNG)

来看这张图，其中红色线是我们的真实标签，而蓝色线是我们的拟合模型。

这是一种比较极端，但的确可能发生的情况。

这张图像上，前半部分的拟合非常成功，看上去我们的真实标签和我们的预测结果几乎重合，但后半部分的拟合却非常糟糕，模型向着与真实标签完全相反的方向去了。

对于这样的一个拟合模型，如果我们使用MSE来对它进行判断，它的MSE会很小，因为大部分样本其实都被完美拟合了，少数样本的真实值和预测值的巨大差异在被均分到每个样本上之后，MSE就会很小。

但这样的拟合结果必然不是一个好结果，因为一旦我的新样本是处于拟合曲线的后半段的，我的预测结果必然会有巨大的偏差，而这不是我们希望看到的。

所以，我们希望找到新的指标，除了判断预测的数值是否正确之外，还能够判断我们的模型是否拟合了足够多的，数值之外的信息。

在我们学习降维算法PCA的时候，我们提到我们使用方差来衡量数据上的信息量。

如果方差越大，代表数据上的信息量越多，而这个信息量不仅包括了数值的大小，还包括了我们希望模型捕捉的那些规律。

为了衡量模型对数据上的信息量的捕捉，我们定义了**$R^2$**来帮助我们：

$$
R^2 = 1 - \frac{\sum_{i=0}^{m}({y_i} - {{\hat{y}_i}})^2}{\sum_{i=0}^{m}({y_i} - \bar{y})^2} = 1 - \frac{RSS}{\sum_{i=0}^{m}({y_i} - \bar{y})^2}
$$

其中$\boldsymbol{y}$是真实标签，$\boldsymbol{\hat{y}}$是预测结果，$\boldsymbol{\bar{y}}$是均值，$y_i - \bar{y}$如果除以样本量m就是我们的方差。

方差的本质是任意一个$y$值和样本均值的差异，差异越大，这些值所带的信息越多。

在$R^2$中，分子是真实值和预测值之差的差值，也就是我们的模型没有捕获到的信息总量，分母是真实标签所带的信息量，所以其衡量的是**1 - 我们的模型没有捕获到的信息量占真实标签中所带的信息量的比例**，所以，$R^2$越接近1越好。

$R^2$可以使用三种方式来调用，一种是直接从metrics中导入r2_score，输入预测值和真实值后打分。

第二种是直接从线性回归LinearRegression的接口score来进行调用。

第三种是在交叉验证中，输入"r2"来调用。

In [32]:
lr.score(X_test, y_test)

0.5926087785518779

In [36]:
r2_score(y_test, y_predict_test)

0.5926087785518779

In [38]:
cross_val_score(lr, X_train, y_train, cv=10, scoring='r2').mean()

0.6062789167942275

我们观察到，在加利福尼亚房屋价值数据集上的MSE其实不是一个很大的数（0.5），但是我们的$R^2$不高，这证明我们的模型比较好地拟合了一部分数据的数值，却没有能正确拟合数据的分布。